In [1]:
from bs4 import BeautifulSoup as bs

import requests
import re


import pandas as pd
import numpy as np
import pyodbc

from datetime import datetime
import time
import random

import os

In [2]:
# this is grabing the 49ers html web page
url = 'https://www.pro-football-reference.com/teams/sfo/2010.htm'

# Table Function

In [3]:
# use this function before creating tables
def tables_html(url:str):
    html = requests.get(url).text
    
    soup = bs(html, 'html.parser')
    
    tables = soup.find_all(class_='table_wrapper')
    
    return tables

In [4]:
tables = tables_html(url=url)

# DF Columns!

In [5]:
def webscrape_columns (tables:str, table_num:int):

    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')
    
    
    #find thead tag to isolate col names
    table_html = table_html.find_all('thead')[0]

    # find class_ = 'poptip' to isolate col names
    col_th_tag = table_html.find_all(class_='poptip')

    # get how many column names there are
    num_col_passing = len(col_th_tag)

    # list of column names
    columns = []

    for i in range(0, num_col_passing):
        try:
            data = col_th_tag[i].contents[0]
            columns.append(data)
        except:
            columns.append('n/a')   
            
    return columns

In [6]:
columns = webscrape_columns(tables=tables, table_num=0)
columns

['Player',
 'PF',
 'Yds',
 'Ply',
 'Y/P',
 'TO',
 'FL',
 '1stD',
 'Cmp',
 'Att',
 'Yds',
 'TD',
 'Int',
 'NY/A',
 '1stD',
 'Att',
 'Yds',
 'TD',
 'Y/A',
 '1stD',
 'Pen',
 'Yds',
 '1stPy',
 '#Dr',
 'Sc%',
 'TO%',
 'Start',
 'Time',
 'Plays',
 'Yds',
 'Pts']

# DF Rows!

## Row

In [7]:
def webscrape_index(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
       
    #get index
    num_index = len(rows_tbody.find_all('th'))
    
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
    return index

In [8]:
index = webscrape_index(tables=tables, table_num=0)
index

['Team Stats', 'Opp. Stats', 'Lg Rank Offense', 'Lg Rank Defense']

In [9]:
def webscrape_rows(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
    

    rows_tr = rows_tbody.find_all('tr')
    
    num_index = len(rows_tbody.find_all('th'))
    num_rows = len(rows_tr[0].find_all('td'))
    
    
    row_data = []

    for i in range(0, num_index):
        row_td = rows_tr[i].find_all('td')
    
        data = []

        for c in range(0, num_rows):
            
            try:
                if len(row_td[c].find_all('a')) > 0:
            
                    d = row_td[c].find_all('a')[0].contents[0]
                    data.append(d)
                
                else:
                    d = row_td[c].contents[0]
                    data.append(d)
            
            except:
                data.append('n/a')
        

        row_data.append(data)
        
        
        
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
        
    #insert index to each row to be equal to all columns
    for i in range(0, len(index)):
        row_data[i].insert(0, index[i])
        
        
    return row_data

In [10]:
rows = webscrape_rows(tables=tables, table_num=0)
rows

[['Team Stats',
  '305',
  '5013',
  '945',
  '5.3',
  '23',
  '8',
  '251',
  '282',
  '500',
  '3356',
  '19',
  '15',
  '6.2',
  '149',
  '401',
  '1657',
  '10',
  '4.1',
  '85',
  '112',
  '933',
  '17',
  '190',
  '26.8',
  '11.1',
  'Own 28.5',
  '2:25',
  '5.12',
  '26.2',
  '1.42'],
 ['Opp. Stats',
  '346',
  '5244',
  '1033',
  '5.1',
  '22',
  '7',
  '299',
  '358',
  '550',
  '3697',
  '25',
  '15',
  '6.3',
  '188',
  '447',
  '1547',
  '9',
  '3.5',
  '86',
  '97',
  '839',
  '25',
  '195',
  '31.8',
  '11.3',
  'Own 30.0',
  '2:35',
  '5.5',
  '26.9',
  '1.65'],
 ['Lg Rank Offense',
  '24',
  '24',
  'n/a',
  'n/a',
  '11',
  '6',
  '30',
  'n/a',
  '22',
  '18',
  '24',
  '13',
  '15',
  'n/a',
  '25',
  '19',
  '21',
  '17',
  'n/a',
  'n/a',
  'n/a',
  'n/a',
  'n/a',
  '26',
  '22',
  '29',
  '25',
  '25',
  '24',
  '27'],
 ['Lg Rank Defense',
  '16',
  '13',
  'n/a',
  'n/a',
  '25',
  '29',
  '15',
  'n/a',
  '22',
  '24',
  '21',
  '17',
  '22',
  'n/a',
  '23',
 

# DF!

In [11]:
def webscrape_tables(tables:str, table_num:int):
    
    table = tables[table_num]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    rows_tbody = table_html.find_all('tbody')[0]
    
    
    #columns
    #find thead tag to isolate col names
    table_html = table_html.find_all('thead')[0]

    # find class_ = 'poptip' to isolate col names
    col_th_tag = table_html.find_all(class_='poptip')

    # get how many column names there are
    num_col_passing = len(col_th_tag)

    # list of column names
    columns = []

    for i in range(0, num_col_passing):
        try:
            data = col_th_tag[i].contents[0]
            columns.append(data)
        except:
            columns.append('n/a') 
    
    
    ##rows
    
    rows_tr = rows_tbody.find_all('tr')
    
    num_index = len(rows_tbody.find_all('th'))
    num_rows = len(rows_tr[0].find_all('td'))
    
    
    row_data = []

    for i in range(0, num_index):
        row_td = rows_tr[i].find_all('td')
    
        data = []

        for c in range(0, num_rows):
            
            try:
                if len(row_td[c].find_all('a')) > 0:
            
                    d = row_td[c].find_all('a')[0].contents[0]
                    data.append(d)
                
                else:
                    d = row_td[c].contents[0]
                    data.append(d)
            
            except:
                data.append('n/a')
        

        row_data.append(data)
        
        
    #index
    index = []
    for i in range(0, num_index):
    
        try:
            data = rows_tbody.find_all('th')[i].contents[0]
            index.append(data)
        except:
            index.append('n/a')
        
        
    #insert index to each row to be equal to all columns
    for i in range(0, len(index)):
        row_data[i].insert(0, index[i])
    
    df = pd.DataFrame(index=index, data=row_data, columns=columns)
    
    return df

In [12]:
df_test = pd.DataFrame(index=index, data=rows, columns=columns)
df_test

,Player,PF,Yds,Ply,Y/P,TO,FL,1stD,Cmp,Att,...,Yds,1stPy,#Dr,Sc%,TO%,Start,Time,Plays,Yds,Pts
Team Stats,Team Stats,305,5013,945,5.3,23,8,251,282,500,...,933,17,190,26.8,11.1,Own 28.5,2:25,5.12,26.2,1.42
Opp. Stats,Opp. Stats,346,5244,1033,5.1,22,7,299,358,550,...,839,25,195,31.8,11.3,Own 30.0,2:35,5.5,26.9,1.65
Lg Rank Offense,Lg Rank Offense,24,24,n/a,n/a,11,6,30,n/a,22,...,n/a,n/a,n/a,26,22,29,25,25,24,27
Lg Rank Defense,Lg Rank Defense,16,13,n/a,n/a,25,29,15,n/a,22,...,n/a,n/a,n/a,15,23,19,16,13,7,14


In [13]:
table_df_li_test = [webscrape_tables(tables, table_num=n) for n in range(0,12)]

In [14]:
[2010] * len(table_df_li_test[0])

[2010, 2010, 2010, 2010]

# Fix Column Labels

In [15]:
#fix column names and addd team name column
def fix_col_labels(table_df_li: list, team: str):
    
    
    #Team Stats and Ranking
    table_df_li[0].columns = [
        'Player', 'Points Scored by Team', 'Total Yds and TO', 'Offensive Plays: Pass Attempts + Rush Attempts + Times Sacked', 
        'Yards per Offensive Play', 'Team Turnovers Lost', 'Fumbles Lost by Player or Team', '1stD', 
        'Passes Completed', 'Passes Attempted', 'Yards Gained by Passing', 'Passing Touchdowns', 
        'Interceptions Thrown', 'Net Yards Gained per Pass Attempt', 'First Downs by Passing',
        'Rushing Attempts', 'Rushing Yards', 'Rushing Touchdowns', 'Rushing Yards per Attempt', 
        'First Downs by Rushing', 'Penalties committed by Team and Accepted','Penalties in Yards Committed by Team', 
        'First Downs by Penalty', 'Number of Drives', 'Per. of Drives Ending in an Offensive Score',
        'Per. of Drives Ending in an Offensive Turnover', 'Average Starting Field Position', 'Average Time per Drive', 
        'Average # of Plays per Drive', 'Net Yards per Drive', 'Average Points Scored per Drive'
    ]
    
    #Schedule and Game Results
    
        #fix col to names
    try:
        table_df_li[1].columns = ['Week', 'Day', 'Date', 'Time', 'n/a', 'Win/Loss', 'Overtime', 'Team Record', 'Home/Away',
                              'Opponent', 'Points Scored', 'Points Allowed', 'Offense 1st Down', 'Total Yards Gained of Offense', 'Total yards Gained by Passing',
                              'Total Yards Gained by Rushing', 'Offense Turnovers', 'Defense 1st Down', 'Total Yards Allowed by Defense',
                              'Total Passing Yards Allowed by Defense', 'Total Rushing Yards Allowed by Defense', 'Defense Turnovers', 
                              'Offense', 'Defense', 'Sp. Tms'
                                 ]

        table_df_li[1].drop(columns=['n/a'], inplace=True)

        
    except:        
        table_df_li[1].columns = ['Week', 'Day', 'Date', 'Time', 'Win/Loss', 'Overtime', 'Team Record', 'Home/Away',
                              'Opponent', 'Points Scored', 'Points Allowed', 'Offense 1st Down', 'Total Yards Gained of Offense', 'Total yards Gained by Passing',
                              'Total Yards Gained by Rushing', 'Offense Turnovers', 'Defense 1st Down', 'Total Yards Allowed by Defense',
                              'Total Passing Yards Allowed by Defense', 'Total Rushing Yards Allowed by Defense', 'Defense Turnovers', 
                              'Offense', 'Defense', 'Sp. Tms'
                                 ]
        # home/away games rows
    table_df_li[1].iloc[:, 7] = ['away' if r == '@' else 'home' for r in table_df_li[1].iloc[:, 7]]          

    
    #Team Conversions
    table_df_li[2].columns = [
        'Player', '3rd Down Attempts in Game', '3rd Down Conversions', '3rd Down Conversion Per.', '4th Down Attempts in Game',
        '4th Down Conversions in Game', '4th Down Conversion Per.', 'Red Zone Attempts', 'Touchdowns Scored After the Team Entered the Red Zone', 
        'Per. of the Time a Team Reaches the Red Zone and Scores a Touchdown'
    ]

    
    #Passing
    table_df_li[3].columns = [
        'No.', 'Player', 'Age', 'Position', 'Games Played', 'Games Started as an Offensive or Defensive Player', 
        'Team Record in Games Stareted by This QB', 'Passes Completed', 'Passes Attemped', 'Per. of Passes Completed', 
        'Yards Gained by Passing', 'Passing Touchdowns', 'Per. of Touchdowns Thrown when Attempting to Pass', 'Interceptions Thrown',
        'Per. of Times Interceped when Attempting to Pass', 'First Downs Passing', 'Passing Success Rate', 'Longest Completed Pass Thrown',
        'Yards Gained per Pass Attempt', 'Adjusted Yards gained per Pass Attempt', 'Yards Gained per Pass Completion', 
        'Yards Gained per Game Played', 'QB Rating', 'ESPN QB Rating', 'Times Sacked', 'Yards Lost due to Sacks', 
        'Per. of Time Sacked when Attempting to Pass', 'Net Yards Gained per Pass Attempt', 'Adjsuted Net Yards per Pass Attempt',
        'Comebacks led by QB', 'Game-winning Drives led by QB'
    ]
    
    
    #Rushing and Receiving
    table_df_li[4].columns = [
        'No.', 'Player', 'Age', 'Position', 'Games Played', 'Games Started as an Offensive or Defensive Player', 'Rushing Attempts',
        'Rushing Yards Gained', 'Rushing Touchdown', 'First Downs Rushing', 'Rushing Success Rate', 'Longest Rushing Attempt', 
        'Rushing Yards per Attempt', 'Rushing Yards per Game', 'Rushing Attempts per Game', 'Pass Targets', 'Receptions', 'Receiving Yards',
        'Receiving Yards per Reception', 'Receiving Touchdowns', 'First Downs Receiving', 'Receiving Success Rate', 'Longest Reception', 
        'Receptions per Game', 'Receiving Yards per Game', 'Catch Per.', 'Receiving Yards per Target', 'Touches: Rushing Attempts and Receptions',
        'Scrimmage Yards per Touch: Rushing + Receiving Yardage per Opportunity', 'Yards from Scrimmage: Receiving and Rushing Yards', 
        'Rushing and Receiving Touchdowns', 'Lost and Recovered Fumbles'
    ]
    
    #kick and punt returns
    table_df_li[5].columns = [
        'No.', 'Player', 'Age', 'Position', 'Games Played', 'Games Started', 'Punts Returned',
        'Punts Return Yardage', 'Punts Returned for Touchdown', 'Longest Punt Return', 
        'Yards per Punt Return', 'Kickoff Returns', 'Yardage for Kickoffs Returned', 
        'Kickoffs Returned for a Touchdown', 'Longest Kickoff Return', 'Yards per Kickoff Return',
        'All-purpose Yards'
    ]
    
    #Kicking
    
    table_df_li[6].columns = [
        'No.', 'Player', 'Age', 'Position', 'Games Played', 'Games Started', 
        'FGA 0-19', 'FGM 0-19', 'FGA 20-29', 'FGM 20-29', 'FGA 30-39', 'FGM 30-39',
        'FGA 40-49', 'FGM 40-49', 'FGA 50+', 'FGM 50+', 'Field Goals Attempted', 'Field Goals Made',
        'Longest Field Goal Made', 'Per. of Field Goals Made', 'Extra Points Attempted', 'Extra Points Made',
        'Extra Point Per.', 'Kickoffs', 'Kickoff Yards', 'Kickoff Touchbacks', 
        'Per. Kickoff was a Touchback', 'Kickoff Average Yardage'
        

    ]
    
    #Punting
    table_df_li[7].columns = [
        'No.', 'Player', 'Age', 'Pos', 'Games Played', 'Games Started', 'Times Puned',
        'Total Punt Yardage', 'Yards per Punt', 'Punt Return Yardage by Opposition', 
        'Punt Net yards', 'Punt Net Yards per Punt', 'Longest Punt', 
        'Punts Resulting in a Touchback', 'Per. of Punts Resulting in a Touchback', 
        'Punts Inside Opp. 20 Yard Line', 'Per. of Punts Downed Inside Opp. 20 Yard Line',
        'Times Punts Blocked'
    ]
    
    #defense and fumbles
    table_df_li[8].columns = [
        'No.','Player', 'Age', 'Pos', 'Games Played', 'Games Started', 'Passes Intercepted on Defense',
        'Yards Interceptions were Returned', 'Interceptions Returned for Touchdowns', 
        'Longest Interception Return', 'Passes Defended by Defensive Player', '# Forced Fumble by Opp.', 
        '# Fumbled both Lost and Recovered by Own Team', 'Fumbles Recovered by Original Fumbler', 
        'Yards Recovered Fumbles were Returned', 'Fumbles Recovered for Touchdown', 'Sacks', 
        'Tackles Solo+Assisted', 'Solo Tackles', 'Assisted Tackles', 'Tackles for Loss', 
        'Quarterback Hits', 'Safeties Scored by Player/Team'
        
    ]
    
    #Scoring Summary
    table_df_li[9].columns = [
         'No.','Player', 'Age', 'Pos', 'Games Played', 'Games Started', 'Rush TD', 'Reception TD', 
        'Punt Return TD', 'Kick Return TD', 'Fumble Return TD', 'Interception TD', 'Other TD', 
        'All Touchdown Scored', '2-Point Conversions Made', 'Two-Point Conversions Attempted', 
        'Defensive Two-Point Conversions', 'Extra Points Made', 'Extra Points Allowed', 'Field Goals Made',
        'Field Goals Attempted', 'Safeties Scored by Player/Team', 'Total Points Scored by all Means',
        'Poins per Game'
    ]
    
    try:
        #touchdown log
        table_df_li[10].drop(columns=['n/a'], inplace=True)
    
        #opponenet touchdown log
        table_df_li[11].drop(columns=['n/a'], inplace=True)
    except:
        pass
    
    #touchdown log
    table_df_li[10].columns = [
        'Rank','Date', 'Opponent', 'Results', 'Quarter', 'Distance', 'Type', 'Detail'
    ]
    
        #opponent touchdown log
    table_df_li[11].columns = [
        'Rank','Date', 'Opponent', 'Results', 'Quarter', 'Distance', 'Type', 'Detail'
    ]

    #insert team name column to each row
    for i in range(0, len(table_df_li)):
        
        num_rows = len(table_df_li[i].values)

        
        table_df_li[i]['Team'] = [team] * len(table_df_li[i])
    
    return table_df_li



In [16]:
#fix inconsistent data types between similar tables
# make all columns in tables the same
##################################################     decorator????     ####################################
def fix_col_dtypes (table_df_li: list, team: str):

    #Team Stats and Rankings
    df = table_df_li[0]

    dtypes_li = ['object', 'int64', 'int64', 'int64', 'float64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'int64', 'int64', 'int64', 'float64', 'int64', 'int64', 'int64', 'int64', 'float64', 'int64', 
                 'int64', 'int64', 'int64', 'int64', 'float64', 'float64', 'object', 'object', 'float64', 'float64', 
                 'float64', 'object']
        
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)
            
    table_df_li[0] = df

    
    #Schedule and Game Results
    df = table_df_li[1] 
    dtypes_li = ['int64','object', 'object', 'object', 'object', 'int64', 'object', 'object', 'object', 'int64', 
                 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'int64', 'float64', 'float64', 'float64', 'object']
    
    for i in range(len(df)):
        try:
            if df['Points Scored'].iloc[i].isnumeric() == False:
                replace_value = df['Points Scored'].iloc[i]
                df[df['Points Scored'] == replace_value] = 0
            else:
                pass
        except:
            pass

    df['Overtime'] = df['Overtime'].fillna(0)
    df['Overtime'] = df['Overtime'].replace('OT', 1)

    df['Week'] = range(1, len(df)+1)

    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[1] = df

    
    #Team Conversions
    df = table_df_li[2] 
    
    try:    
        df['3rd Down Conversion Per.'] = df['3rd Down Conversion Per.'].str.strip('%')
        df['4th Down Conversion Per.'] = df['4th Down Conversion Per.'].str.strip('%')
        df['Per. of the Time a Team Reaches the Red Zone and Scores a Touchdown'] = \
        df['Per. of the Time a Team Reaches the Red Zone and Scores a Touchdown'].str.strip('%')

    except:
        pass
            
    dtypes_li = ['object', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
                 'float64', 'object']
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[2] = df

    
    #Passing
    df = table_df_li[3]
    
    dtypes_li = ['object', 'object', 'int64', 'object', 'int64', 'int64', 'object', 'int64', 'int64', 
                 'float64', 'int64', 'int64', 'float64', 'int64', 'float64', 'int64', 'float64', 
                 'int64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 
                 'int64', 'int64', 'float64', 'float64', 'float64', 'int64', 'int64', 'object']
        
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[3] = df

    
    #Rushing and Receiving
    df = table_df_li[4]
    
    dtypes_li = ['object', 'object', 'int64', 'object', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'float64', 'int64', 'float64', 'float64', 'float64', 'int64', 'int64', 'int64', 'float64','int64', 
                 'int64', 'float64', 'int64', 'float64', 'float64', 'float64', 'float64', 'int64', 'float64','int64',
                 'int64', 'int64', 'object']

    df['Catch Per.'] = df['Catch Per.'].str.strip('%')
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[4] = df

    
    #kick and punt returns
    df = table_df_li[5]
    
    dtypes_li = ['object', 'object', 'int64', 'object', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'float64', 'int64', 'int64', 'int64', 'int64', 'float64', 'int64', 'object']

    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[5] = df

    
    #Kicking                         
    df = table_df_li[6] 

    df['FGA 0-19'] = df['FGA 0-19'].fillna(0)
    df['FGA 20-29'] = df['FGA 20-29'].fillna(0)
    df['FGA 30-39'] = df['FGA 30-39'].fillna(0)
    df['FGA 40-49'] = df['FGA 40-49'].fillna(0)
    df['FGA 50+'] = df['FGA 50+'].fillna(0)

    df['FGM 0-19'] = df['FGM 0-19'].fillna(0)
    df['FGM 20-29'] = df['FGM 20-29'].fillna(0)
    df['FGM 30-39'] = df['FGM 30-39'].fillna(0)
    df['FGM 40-49'] = df['FGM 40-49'].fillna(0)
    df['FGM 50+'] = df['FGM 50+'].fillna(0)

    df['Field Goals Attempted'] = df['Field Goals Attempted'].fillna(0)
    df['Field Goals Made'] = df['Field Goals Made'].fillna(0)
    df['Longest Field Goal Made'] = df['Longest Field Goal Made'].fillna(0)
    
    df['Per. of Field Goals Made'] = df['Per. of Field Goals Made'].str.strip('%') 
    df['Per. of Field Goals Made'] = df['Per. of Field Goals Made'].fillna(0)

    df['Extra Points Attempted'] = df['Extra Points Attempted'].fillna(0)
    df['Extra Points Made'] = df['Extra Points Made'].fillna(0)

    df['Extra Point Per.'] = df['Extra Point Per.'].str.strip('%')
    df['Extra Point Per.'] = df['Extra Point Per.'].fillna(0)

    df['Kickoffs'] = df['Kickoffs'].fillna(0)
    df['Kickoff Yards'] = df['Kickoff Yards'].fillna(0)

    df['Kickoff Touchbacks'] = df['Kickoff Touchbacks'].fillna(0)

    df['Per. Kickoff was a Touchback'] = df['Per. Kickoff was a Touchback'].str.strip('%')
    df['Per. Kickoff was a Touchback'] = df['Per. Kickoff was a Touchback'].fillna(0)

    df['Kickoff Average Yardage'] = df['Kickoff Average Yardage'].fillna(0)
    
    dtypes_li = ['int64', 'object', 'int64', 'object', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'float64', 
                 'int64', 'int64', 'float64','int64', 'int64', 'int64', 'float64', 'float64', 'object']
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[6] = df

    
    #Punting
    df = table_df_li[7]

    dtypes_li = ['object', 'object', 'int64', 'object', 'int64', 'int64', 'int64', 'int64', 'float64', 'int64',
                 'int64', 'float64', 'int64', 'int64', 'float64', 'int64', 'float64', 'int64', 'object']
    
    df['Per. of Punts Resulting in a Touchback'] = df['Per. of Punts Resulting in a Touchback'].str.strip('%')
    df['Per. of Punts Downed Inside Opp. 20 Yard Line'] = df['Per. of Punts Downed Inside Opp. 20 Yard Line'].str.strip('%')
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[7] = df

    
    #defense and fumbles
    df = table_df_li[8]
    
    dtypes_li = ['object', 'object', 'int64', 'object', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'float64', 'int64', 'int64', 'int64', 
                 'int64', 'int64', 'int64', 'object']

    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[8] = df

    
    #Scoring Summary
    df = table_df_li[9]

    dtypes_li = ['object', 'object', 'int64', 'object', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 'int64', 
                 'int64', 'object', 'int64', 'float64', 'object']
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[9] = df
    

    #Touchdown Log
    df = table_df_li[10]

    dtypes_li = ['object', 'object', 'object', 'object', 'object', 'int64', 'object', 'object', 'object']
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[10] = df

    
    #Opponent
    df = table_df_li[11]
    
    dtypes_li = ['object', 'object', 'object', 'object', 'object', 'int64', 'object', 'object', 'object']
    
    for col, dtype in zip(df.columns,dtypes_li):
        try:
            df[col] = df[col].replace('n/a', 0)
            df[col] = df[col].astype(dtype)
        except:
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(dtype)
        finally:
            df[col] = df[col].astype(dtype)

    table_df_li[11] = df

    
    return table_df_li

In [17]:
#convert time to 24 hours
def convert_to_24hour(time_str):
    hour, minute, am_pm = re.findall('\d+|\w+', time_str)
    hour = int(hour)
    if am_pm == 'PM' and hour != 12:
        hour += 12
    elif am_pm == 'AM' and hour == 12:
        hour = 0
    return f'{hour:02d}:{minute}'

In [18]:
#fix data columns, splits, dates, feature engineering

def fix_data(table_df_li: list, team: str):
    #team_stats_and_ranking - ok

    #schedule_and_game_results
        #delete extra rows
    df = table_df_li[1]
    
    df = df[df['Team'] != 0]
    df = df.reset_index(drop=True)
    
        #fix time
    time_dict = {'Time': []}
    fixed_date_dict = {'Date': []}
    for i in df.index:
        try:
            time_dict['Time'].append(convert_to_24hour(df['Time'].iloc[i]))
        except:
            time_dict['Time'].append('nan')

        try:
            if ('January' in df['Date'].iloc[i]) or ('February' in df['Date'].iloc[i]):
                mon_day = df['Date'].iloc[i]
                y = df['Year'].iloc[i]+1
            
                date = mon_day +' '+ str(y)
                
                dt = datetime.strptime(date, '%B %d %Y')
            
                fixed_date = datetime.strftime(dt, format='%m/%d/%Y')
        
                fixed_date = datetime.strptime(fixed_date, '%m/%d/%Y')
        
            
                fixed_date_dict['Date'].append(fixed_date)
                
            else:
                mon_day = df['Date'].iloc[i]
                y = df['Year'].iloc[i]
            
                date = mon_day +' '+ str(y)
                
                dt = datetime.strptime(date, '%B %d %Y')
            
                fixed_date = datetime.strftime(dt, format='%m/%d/%Y')
        
                fixed_date = datetime.strptime(fixed_date,'%m/%d/%Y')
        
            
                fixed_date_dict['Date'].append(fixed_date)
    
        except:
            fixed_date = 'nan'
            fixed_date_dict['Date'].append(fixed_date)

    
    df['Time'] = time_dict['Time']
    df['Date'] = fixed_date_dict['Date']

    df['Win/Loss'] = df['Win/Loss'].replace('L', 'Loss')
    df['Win/Loss'] = df['Win/Loss'].replace('W', 'Win')

    table_df_li[1] = df

    #team_conversions - ok

    #passing - ok

    #rushing_and_receiving - ok

    #kick_and_punt_returns - ok

    #kicking - ok

    #punting - ok

    #defense_and_fumbles - ok

    #scoring_summary - ok

    
    #touchdown_log
    df = table_df_li[10]

    win_loss_li = []
    home_score_li = []
    opp_score_li = []
    
    for i in range(0, len(df)):
        try:
            df_res_split = df['Results'].iloc[i].split(',')
    
            if df_res_split[0] == 'L':
                win_loss = df_res_split[0].replace('L', 'Loss')
                win_loss_li.append(win_loss)

            else:
                win_loss = df_res_split[0].replace('W', 'Win')
                win_loss_li.append(win_loss)

            hs, os = df_res_split[1].split('-')

            hs = hs.strip()
            os = os.strip()

            home_score_li.append(hs)
            opp_score_li.append(os)
        
        except:
            win_loss_li.append('nan')
            home_score_li.append('nan')
            opp_score_li.append('nan')

    df['Win/Loss'] = win_loss_li
    df['Home_Score'] = home_score_li
    df['Opponenet_Score'] = opp_score_li

    df = df.drop(columns=['Results'])

    df = df[['Rank', 'Team', 'Date', 'Opponent', 'Win/Loss', 'Home_Score', 'Opponenet_Score', 'Quarter', 'Distance', 'Type', 'Detail']]

    table_df_li[10] = df
    

    #opponent touchdown log
    df = table_df_li[11]

    win_loss_li = []
    home_score_li = []
    opp_score_li = []
    
    for i in range(0, len(df)):
        try:
            df_res_split = df['Results'].iloc[i].split(',')
    
            if df_res_split[0] == 'L':
                win_loss = df_res_split[0].replace('L', 'Loss')
                win_loss_li.append(win_loss)

            else:
                win_loss = df_res_split[0].replace('W', 'Win')
                win_loss_li.append(win_loss)
                
            hs, os = df_res_split[1].split('-')

            hs = hs.strip()
            os = os.strip()

            home_score_li.append(hs)
            opp_score_li.append(os)
        
        except:
            win_loss_li.append('nan')
            home_score_li.append('nan')
            opp_score_li.append('nan')
            

    df['Win/Loss'] = win_loss_li
    df['Home_Score'] = home_score_li
    df['Opponenet_Score'] = opp_score_li

    df = df.drop(columns=['Results'])

    df = df[['Rank','Team', 'Date', 'Opponent', 'Win/Loss', 'Home_Score', 'Opponenet_Score', 'Quarter', 'Distance', 'Type', 'Detail']]


    table_df_li[11] = df

    
    return table_df_li

# Final Save CSV Function

In [19]:
def webscrape_create_dataframes_save_csv(team: str, year: int):    
    
    team_to_url_df = pd.DataFrame(
        data=[
            [
                'crd', #Cardinals
                'atl', #Falcons
                'rav', #Ravens
                'buf', #Buffalos
                'car', #Panthers
                'chi', #Bears
                'cin', #Bengals
                'cle', #Browns
                'dal', #Cowboys
                'den', #Broncos
                'det', #Lions
                'gnb', #Packers
                'htx', #Texans
                'clt', # Colts
                'jax', #Jaguars
                'kan', #Chiefs
                'rai', #Raiders
                'sdg', #Chargers
                'ram', #Rams
                'mia', #Dolphins
                'min', #Vikings
                'nwe', #Patriots
                'nor', #Saints
                'nyg', #Giants
                'nyj', #Jets
                'phi', #Eagles
                'pit', #Steelers
                'sfo', #49ers
                'sea', #Seahawks
                'tam', #Buccaneers
                'oti', #Titans
                'was' #Commanders
            ]
        ], 
        columns=[
            'Cardinals',
            'Falcons',
            'Ravens',
            'Buffalos',
            'Panthers',
            'Bears',
            'Bengals',
            'Browns',
            'Cowboys',
            'Broncos',
            'Lions',
            'Packers',
            'Texans',
            'Colts',
            'Jaguars',
            'Chiefs',
            'Raiders',
            'Chargers',
            'Rams',
            'Dolphins',
            'Vikings',
            'Patriots',
            'Saints',
            'Giants',
            'Jets',
            'Eagles',
            'Steelers',
            '49ers',
            'Seahawks',
            'Buccaneers',
            'Titans',
            'Commanders'
        ])
    
    # string url
    str_url = 'https://www.pro-football-reference.com/teams/{}/{}.htm'

    #get team url ID
    team_url_str = team_to_url_df[team].values[0]
    
    # URL
    url = str_url.format(team_url_str, year)
    
    # request html text
    tables_html_request = tables_html(url=url)
    
    # delay execute so not to trigger ip address block
    time.sleep(random.randrange(7,9))
    
    # webscrape tables/ create dataframes
    
    # table name list with team and year to format
    table_name_li = [
        'team_stats_and_ranking_{}_{}',
        'schedule_and_game_results_{}_{}',
        'team_conversions_{}_{}',
        'passing_{}_{}',
        'rushing_and_receiving_{}_{}',
        'kick_and_punt_returns_{}_{}',
        'kicking_{}_{}',
        'punting_{}_{}',
        'defense_and_fumbles_{}_{}',
        'scoring_summary_{}_{}',
        'touchdown_log_{}_{}',
        'opponent_touchdown_log_{}_{}'
    ]
    
    #file location
    file_loc = 'NFL_Data_{}/{}/'.format(year, team)
    
    table_num = 12

    #put tables into a list
    table_df_li = [webscrape_tables(tables=tables_html_request, table_num=i) for i in range(0,table_num)]

    #use list of tabels to fix all the columns
    table_df_li = fix_col_labels(table_df_li=table_df_li, team=team)
    table_df_li = fix_col_dtypes(table_df_li=table_df_li, team=team)

    #upload tables to csv file
    for t in range(0, table_num):

        #add the year to each row
        table_df_li[t]['Year'] = [year] * len(table_df_li[t])

    
    #fix data on tables
    table_df_li = fix_data(table_df_li=table_df_li, team=team)

    #import to each year and team file
    for t in range(0, table_num):   
        table_df_li[t].to_csv(file_loc+table_name_li[t].format(team, year))
        
    return  table_df_li
        
        

# Download CSV Files from all teams 2010-2023

In [20]:
years = range(2010,2024)

In [21]:
team_names=[
    '49ers',
     'Bears',
     'Bengals',
     'Broncos',
     'Browns',
     'Buccaneers',
     'Buffalos',
     'Cardinals',
     'Chargers',
     'Chiefs',
     'Colts',
     'Commanders',
     'Cowboys',
     'Dolphins',
     'Eagles',
     'Falcons',
     'Giants',
     'Jaguars',
     'Jets',
     'Lions',
     'Packers',
     'Panthers',
     'Patriots',
     'Raiders',
     'Rams',
     'Ravens',
     'Saints',
     'Seahawks',
     'Steelers',
     'Texans',
     'Titans',
     'Vikings'
]

In [22]:
table_name_li = [
    'team_stats_and_ranking_{}_{}',
    'schedule_and_game_results_{}_{}',
    'team_conversions_{}_{}',
    'passing_{}_{}',
    'rushing_and_receiving_{}_{}',
    'kick_and_punt_returns_{}_{}',
    'kicking_{}_{}',
    'punting_{}_{}',
    'defense_and_fumbles_{}_{}',
    'scoring_summary_{}_{}',
    'touchdown_log_{}_{}',
    'opponent_touchdown_log_{}_{}'
]

In [23]:

#this will webscrape all the teams tables from 2010-2022
#do not run 
###                         45-60 min run
for year in years:
    for team in team_names:
        print(team, year)
        webscrape_create_dataframes_save_csv(team=team, year=year)
        
        

49ers 2010
Bears 2010
Bengals 2010
Broncos 2010
Browns 2010
Buccaneers 2010
Buffalos 2010
Cardinals 2010
Chargers 2010
Chiefs 2010
Colts 2010
Commanders 2010
Cowboys 2010
Dolphins 2010
Eagles 2010
Falcons 2010
Giants 2010
Jaguars 2010
Jets 2010
Lions 2010
Packers 2010
Panthers 2010
Patriots 2010
Raiders 2010
Rams 2010
Ravens 2010
Saints 2010
Seahawks 2010
Steelers 2010
Texans 2010
Titans 2010
Vikings 2010
49ers 2011
Bears 2011
Bengals 2011
Broncos 2011
Browns 2011
Buccaneers 2011
Buffalos 2011
Cardinals 2011
Chargers 2011
Chiefs 2011
Colts 2011
Commanders 2011
Cowboys 2011
Dolphins 2011
Eagles 2011
Falcons 2011
Giants 2011
Jaguars 2011
Jets 2011
Lions 2011
Packers 2011
Panthers 2011
Patriots 2011
Raiders 2011
Rams 2011
Ravens 2011
Saints 2011
Seahawks 2011
Steelers 2011
Texans 2011
Titans 2011
Vikings 2011
49ers 2012
Bears 2012
Bengals 2012
Broncos 2012
Browns 2012
Buccaneers 2012
Buffalos 2012
Cardinals 2012
Chargers 2012
Chiefs 2012
Colts 2012
Commanders 2012
Cowboys 2012
Dolphins 20

#for current year
for year in [2023]:
    for team in team_names:
        webscrape_create_dataframes_save_csv(team=team, year=year)

In [24]:
pd.read_csv('NFL_Data_2010/49ers/schedule_and_game_results_49ers_2010')

,Unnamed: 0,Week,Day,Date,Time,Win/Loss,Overtime,Team Record,Home/Away,Opponent,...,Defense 1st Down,Total Yards Allowed by Defense,Total Passing Yards Allowed by Defense,Total Rushing Yards Allowed by Defense,Defense Turnovers,Offense,Defense,Sp. Tms,Team,Year
0,0,1,Sun,2010-09-12,NaN,Loss,0,0-1,away,Seattle Seahawks,...,14,242,165,77,1,-21.98,-2.30,-2.49,49ers,2010
1,1,2,Mon,2010-09-20,NaN,Loss,0,0-2,home,New Orleans Saints,...,17,287,237,50,0,12.11,-2.64,-14.08,49ers,2010
2,2,3,Sun,2010-09-26,NaN,Loss,0,0-3,away,Kansas City Chiefs,...,21,457,250,207,1,-13.14,-14.03,6.17,49ers,2010
3,3,4,Sun,2010-10-03,NaN,Loss,0,0-4,away,Atlanta Falcons,...,23,357,259,98,2,-0.55,-2.45,1.94,49ers,2010
4,4,5,Sun,2010-10-10,NaN,Loss,0,0-5,home,Philadelphia Eagles,...,17,352,227,125,1,-8.09,0.80,2.65,49ers,2010
5,5,6,Sun,2010-10-17,NaN,Win,0,1-5,home,Oakland Raiders,...,10,179,69,110,2,1.59,12.76,-3.84,49ers,2010
6,6,7,Sun,2010-10-24,NaN,Loss,0,1-6,away,Carolina Panthers,...,22,379,303,76,3,-5.10,7.77,-2.32,49ers,2010
7,7,8,Sun,2010-10-31,NaN,Win,0,2-6,home,Denver Broncos,...,22,398,339,59,2,6.70,-1.65,0.43,49ers,2010
8,8,10,Sun,2010-11-14,NaN,Win,1,3-6,home,St. Louis Rams,...,22,332,228,104,0,9.50,2.36,-6.66,49ers,2010
9,9,11,Sun,2010-11-21,NaN,Loss,0,3-7,home,Tampa Bay Buccaneers,...,20,299,137,162,0,-23.69,-4.42,4.19,49ers,2010


In [25]:
stop

NameError: name 'stop' is not defined